# Vulnerable Spring

[vuln-spring](https://github.com/HooliCorp/vuln-spring) is an intentionally vulnerable spring application to test SAST tools. The purpose of this notebook is to demonstrate detecting the various vulnerabilities using joern's CPGQL queries.

We begin by setting up the basics for joern.

In [1]:
import asyncio

from joern_lib import client, workspace
from joern_lib.detectors import common as cpg, java
from joern_lib.utils import print_table, print_tree, print_flows

joern_host = "http://joern:9000"
cpggen_host = "http://cpggen:7072"
joern_username = "admin"
joern_password  = "admin"

## Generate CPG using cpggen

To improve performance, CPG for this project is generated using the cpggen server and then imported onto the joern server. The default docker-compose starts a cpggen server with the hostname `cpggen` and port `7072`.

We use the `create_cpg` api from workspace to create a CPG from the git repo directly. This single call would handle both generation of the cpg and import onto joern. While languages parameter is optional, we specify the language `java-with-deps` which improves the method full names by inferring types.

In [3]:
async def generate_cpg():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await workspace.create_cpg(connection, "https://github.com/HooliCorp/vuln-spring", out_dir="/tmp/vuln-spring/cpg_out", languages="java", project_name="vuln-spring-java")
    print (res)
asyncio.run(generate_cpg())

True


## Inspect methods

With a fresh CPG imported, one of the initial activities to perform as a threat hunter is to inspect the methods in the application. Visually inspecting the methods, would help understand not just the custom code but also the frameworks and libraries used.

joern-lib offers a convenient api called `list_methods` to simplify the inspection. You can invoke this method without arguments to receive a full list of methods or provide a regex to narrow the results.

In [5]:
async def inspect_methods():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_methods(connection)
    print_table(res, title="vuln-spring methods")
asyncio.run(inspect_methods())

                                                                                                                                  vuln-spring methods                                                                                                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name                              ┃ fullName                           ┃ signature                          ┃ lineNumberEnd ┃ _label ┃ code                               ┃ isExternal ┃ lineNumber ┃                                id ┃ order ┃ filename                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━

In [6]:
async def search_methods():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_methods(connection, ".*vulnspring.*")
    print_table(res, title="Methods Search Result")
asyncio.run(search_methods())

                                                                                                                                 Methods Search Result                                                                                                                                  
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name              ┃ fullName                              ┃ signature                             ┃ lineNumberEnd ┃ _label ┃ code                                  ┃ isExternal ┃ lineNumber ┃                                    id ┃ order ┃ filename                              ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Inspect annotations

Inspecting annotations is a quicker way to identify the various http routes, components, and controllers in web applications. You can use the `list_annotations` api from cpg.

In [8]:
async def inspect_annotations():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.list_annotations(connection)
    print_table(res, title="vuln-spring annotations")
asyncio.run(inspect_annotations())

                                                                                                                                vuln-spring annotations                                                                                                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ name                         ┃ code                                                                                            ┃   order ┃ _label         ┃     argumentIndex ┃ fullName                                                                     ┃    lineNumber ┃    id ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━╇

## Identify complex methods

Complex methods typically contain a number of control structures. A convenient function called `get_complex_functions` could be used with the `n` argument to identify such methods of interest.

In [10]:
async def inspect_complex_methods():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.get_complex_methods(connection, n=3)
    print_table(res, title="vuln-spring complex methods")
asyncio.run(inspect_complex_methods())

                                                                                                                              vuln-spring complex methods                                                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ packageName            ┃ symbol       ┃ filename                                 ┃ className                            ┃ methodFullName                           ┃ lineNumber ┃ classShortName ┃ methodShortName ┃ node                                     ┃ _label   ┃ nodeLabel ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Get the call tree

The next step in our journey is to visualize the call tree for some interesting methods to dig deeper.

In [12]:
async def get_call_tree():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await cpg.get_call_tree(connection, "com.example.vulnspring.WebController.login:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,java.lang.String,org.springframework.ui.Model)")
    print_tree(res)
asyncio.run(get_call_tree())

com.example.vulnspring.WebController.login:java.lang.String(javax.servlet.http.HttpSession,java.lang.String,java.lang.String,org.springframework.ui.Model)
┣━━ org.slf4j.Logger.debug:void(java.lang.String)
┣━━ com.example.vulnspring.WebController.loginSuccess:boolean(java.lang.String,java.lang.String)
┣━━ org.springframework.jdbc.core.JdbcTemplate.queryForMap:java.util.Map(java.lang.String)
┣━━ |    java.util.Map.containsKey:boolean(java.lang.Object)
┣━━ |    |    org.slf4j.Logger.debug:void(java.lang.String)
┗━━ |    |    |    javax.servlet.http.HttpSession.setAttribute:void(java.lang.String,java.lang.Object)


## List HTTP routes and filters

joern-lib comes packaged with useful api methods to simplify code analysis. `list_http_routes` and `list_http_filters` are starter methods to list some HTTP routes and filters (sources) for the application.

In [14]:
async def get_routes():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await java.list_http_routes(connection)
    print_table(res, title="vuln-spring http routes")
asyncio.run(get_routes())

                                                                                                                                vuln-spring http routes                                                                                                                                 
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name              ┃ fullName                             ┃ signature                           ┃ lineNumberEnd ┃ _label ┃ code                                 ┃ isExternal ┃ lineNumber ┃   id ┃ order ┃ filename                            ┃ annotation                           ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
async def get_filters():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    res = await java.list_http_filters(connection)
    print_table(res, title="vuln-spring web filters")
asyncio.run(get_filters())

                                                                                                                                vuln-spring web filters                                                                                                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ inheritsFromTypeFullName                     ┃ name          ┃ fullName                             ┃ _label    ┃ code                       ┃ isExternal ┃ lineNumber ┃   id ┃ order ┃ filename                                      ┃ annotation                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇

# OWASP Top 10

Having mastered the basics of Joern and joern-lib, it is time to dive into detecting OWASP top 10 vulnerabilities in this project. To make life easy, we first tag all spring request parameters as `attacker-controlled`. This would help simplify future data-flow queries.

In [17]:
async def create_attacker_controlled_tag():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await cpg.create_tags(connection, query='cpg.annotation.fullName("org.springframework.web.bind.annotation.Request.*").parameter.typeFullNameExact("java.lang.String")', tags=["attacker-controlled"])
    res = await cpg.list_tags(connection, name="attacker-controlled", is_parameter=True)
    print_table(res)
asyncio.run(create_attacker_controlled_tag())

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┓
┃ dynamicTypeHintFullName        ┃ name               ┃ evaluationStrategy       ┃ isVariadic     ┃ code                                                                                ┃ typeFullName          ┃   order ┃ _label                    ┃   index ┃   lineNumber ┃    id ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━┩
│                                │ username           │ BY_SHARING               │ False          │ @RequestParam(name = "username", required = true) String 

## A01:2021 – Broken Access Control

Directory Traversal vulnerability exists when an attacker-controlled parameter is passed to a file read operation.

In [19]:
async def detect_traversal():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullNameExact("java.io.File.<init>:void(java.lang.String)").parameter.order(1)')
asyncio.run(detect_traversal())

src/main/java/com/example/vulnspring/WebController.java:119 @RequestParam(name = "dbpath") String dbpath
├── src/main/java/com/example/vulnspring/WebController.java:121 FilenameUtils.getBaseName(dbpath)
├── src/main/java/com/example/vulnspring/WebController.java:131 sanitizedValue + ".bak"
└── java.io.File( p1 )



Sensitive data exposure arises when a sensitive data such as password or PII data is exposed to an external service or logging method without obfuscation.

In [21]:
async def detect_sde():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    # Tag certain parameters as sensitive
    await cpg.create_tags(connection, query='cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.name("(?i).*(password|token|key|secret).*")', tags=["sensitive"])
    # Query based on the tag
    await client.df(connection, {"parameter": "sensitive"}, 'cpg.method.fullName("org.slf4j.Logger.*").parameter.orderGt(0)')
asyncio.run(detect_sde())

src/main/java/com/example/vulnspring/WebController.java:59 @RequestParam(name = "password", required = true) String password
├── src/main/java/com/example/vulnspring/WebController.java:69 String password
├── src/main/java/com/example/vulnspring/WebController.java:61 "Login with: " + username + ":" + password
└── org.slf4j.Logger.debug( p1 )



JWT encode/decode misconfigurations

In [23]:
async def detect_jwt_issues():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.typeDecl.fullName("com.auth0.jwt.*").method.parameter')
    print("*** WARNING: JWT none algorithm used here ***\n")
    # Find usages of JWT none algorithm
    await client.df(connection, 'cpg.typeDecl.fullName("com.auth0.jwt.algorithms.Algorithm").method.name("none").methodReturn', 'cpg.typeDecl.fullName("com.auth0.jwt.*").method.parameter.order(1)')
asyncio.run(detect_jwt_issues())

src/main/java/com/example/vulnspring/WebController.java:287 @RequestParam String jwtString
├── src/main/java/com/example/vulnspring/WebController.java:288 JWT.decode(jwtString)
└── com.auth0.jwt.interfaces.DecodedJWT.getClaim( p0 )

src/main/java/com/example/vulnspring/WebController.java:287 @RequestParam String jwtString
├── src/main/java/com/example/vulnspring/WebController.java:288 JWT.decode(jwtString)
├── src/main/java/com/example/vulnspring/WebController.java:291 decodedJWT.getClaim("username")
└── com.auth0.jwt.interfaces.Claim.asString( p0 )

src/main/java/com/example/vulnspring/WebController.java:287 @RequestParam String jwtString
└── com.auth0.jwt.JWT.decode( p1 )

*** WARNING: JWT none algorithm used here ***

src/main/java/com/example/vulnspring/WebController.java:279 Algorithm.none()
└── com.auth0.jwt.JWTCreator$Builder.sign( p1 )



Open redirect

In [25]:
async def detect_open_redirect():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.call(Operators.addition).code(".*redirect.*").method.methodReturn')
asyncio.run(detect_open_redirect())

src/main/java/com/example/vulnspring/WebController.java:259 @RequestParam(required = false) String desk
└── src/main/java/com/example/vulnspring/WebController.java:263 "redirect:" + helpDeskLocation



## A03:2021 – Injection

### SQL Injection

The first style of SQL Injection we will detect with joern is an end-to-end example where attacker-controlled string parameter is used directly in a query without parameterization.

It is important to limit only to string parameter since numeric values cannot result in a successful injection. We first show the implementation using the `RequestParam` parameter.

In [27]:
async def detect_sqli():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, 'cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.typeFullNameExact("java.lang.String")',
        'cpg.typeDecl.fullName("org.springframework.jdbc.core.JdbcTemplate").method.parameter.index(0)')
asyncio.run(detect_sqli())

src/main/java/com/example/vulnspring/WebController.java:103 @RequestParam(name = "newname") String newName
├── src/main/java/com/example/vulnspring/WebController.java:105 new Object[] { newName, session.getAttribute("username") }
├── src/main/java/com/example/vulnspring/WebController.java:105 this.jdbcTemplate
└── org.springframework.jdbc.core.JdbcTemplate.update( p0 )

src/main/java/com/example/vulnspring/WebController.java:154 @RequestParam(name = "toaccount") String toAccount
├── src/main/java/com/example/vulnspring/WebController.java:172 new Object[] { toAccount }
├── src/main/java/com/example/vulnspring/WebController.java:171 this.jdbcTemplate
├── src/main/java/com/example/vulnspring/WebController.java:182 this.jdbcTemplate
└── org.springframework.jdbc.core.JdbcTemplate.queryForMap( p0 )

src/main/java/com/example/vulnspring/WebController.java:58 @RequestParam(name = "username", required = true) String username
├── src/main/java/com/example/vulnspring/WebController.java:69 String 

The source parameter in the above example could be rewritten to use the custom `attacker-controlled` tag that we created. This style is adopted in the rest of the exercise.

In [29]:
async def detect_sqli_attacker_controlled():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.typeDecl.fullName("org.springframework.jdbc.core.JdbcTemplate").method.parameter.order(1)')
asyncio.run(detect_sqli_attacker_controlled())

src/main/java/com/example/vulnspring/WebController.java:154 @RequestParam(name = "toaccount") String toAccount
├── src/main/java/com/example/vulnspring/WebController.java:172 new Object[] { toAccount }
├── src/main/java/com/example/vulnspring/WebController.java:171 this.jdbcTemplate
├── src/main/java/com/example/vulnspring/WebController.java:182 jdbcTemplate.queryForMap(fromAccountStatement, new Object[] { session.getAttribute("username") })
├── src/main/java/com/example/vulnspring/WebController.java:185 (float) fromResultMap.get("balance")
├── src/main/java/com/example/vulnspring/WebController.java:205 new Object[] { fromAccountBalance - amount, fromAccount }
├── src/main/java/com/example/vulnspring/WebController.java:204 this.jdbcTemplate
└── org.springframework.jdbc.core.JdbcTemplate.update( p1 )

src/main/java/com/example/vulnspring/WebController.java:59 @RequestParam(name = "password", required = true) String password
├── src/main/java/com/example/vulnspring/WebController.java:69 

The sink query could be rewritten to use a tag called `sql-sink`

In [31]:
async def create_sql_sink_tag():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await cpg.create_tags(connection, query='cpg.typeDecl.fullName("org.springframework.jdbc.core.JdbcTemplate").method.parameter.order(1)', tags=["sql-sink"])
    res = await cpg.list_tags(connection, name="sql-sink", is_parameter=True)
    print_table(res)
asyncio.run(create_sql_sink_tag())

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ dynamicTypeHintFullName                               ┃ name        ┃ evaluationStrategy                         ┃ isVariadic               ┃ code        ┃ typeFullName                  ┃         order ┃ _label                                      ┃         index ┃         id ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│                                                       │ p1          │ BY_VALUE                                   │ False                    │ p1          │

In [32]:
async def detect_sqli_taggged():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, {"parameter": "sql-sink"})
asyncio.run(detect_sqli_taggged())

src/main/java/com/example/vulnspring/WebController.java:154 @RequestParam(name = "toaccount") String toAccount
├── src/main/java/com/example/vulnspring/WebController.java:172 new Object[] { toAccount }
├── src/main/java/com/example/vulnspring/WebController.java:171 this.jdbcTemplate
├── src/main/java/com/example/vulnspring/WebController.java:182 jdbcTemplate.queryForMap(fromAccountStatement, new Object[] { session.getAttribute("username") })
├── src/main/java/com/example/vulnspring/WebController.java:185 (float) fromResultMap.get("balance")
├── src/main/java/com/example/vulnspring/WebController.java:205 new Object[] { fromAccountBalance - amount, fromAccount }
├── src/main/java/com/example/vulnspring/WebController.java:204 this.jdbcTemplate
└── org.springframework.jdbc.core.JdbcTemplate.update( p1 )

src/main/java/com/example/vulnspring/WebController.java:59 @RequestParam(name = "password", required = true) String password
├── src/main/java/com/example/vulnspring/WebController.java:69 

In [33]:
async def detect_reflected_xss():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, 'cpg.annotation.fullNameExact("org.springframework.web.bind.annotation.RequestParam").parameter.typeFullNameExact("java.lang.String")', 'cpg.typeDecl.fullName("org.springframework.ui.Model").method.parameter.order(2)', filter=['skip_checks'])
asyncio.run(detect_reflected_xss())

src/main/java/com/example/vulnspring/WebController.java:154 @RequestParam(name = "toaccount") String toAccount
├── src/main/java/com/example/vulnspring/WebController.java:172 new Object[] { toAccount }
├── src/main/java/com/example/vulnspring/WebController.java:171 this.jdbcTemplate
├── src/main/java/com/example/vulnspring/WebController.java:182 jdbcTemplate.queryForMap(fromAccountStatement, new Object[] { session.getAttribute("username") })
├── src/main/java/com/example/vulnspring/WebController.java:185 (float) fromResultMap.get("balance")
├── src/main/java/com/example/vulnspring/WebController.java:189 fromAccountBalance - amount
└── org.springframework.ui.Model.addAttribute( p2 )

src/main/java/com/example/vulnspring/WebController.java:103 @RequestParam(name = "newname") String newName
└── org.springframework.ui.Model.addAttribute( p2 )



## A05:2021 – Security Misconfiguration

XXE

In [35]:
async def detect_xxe():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.fullName("javax.xml.parsers.DocumentBuilder.*").parameter.order(1)')
asyncio.run(detect_xxe())

src/main/java/com/example/vulnspring/WebController.java:227 @RequestBody String body
└── javax.xml.parsers.DocumentBuilder.parse( p1 )



## A10:2021 – Server-Side Request Forgery (SSRF)

In [37]:
async def detect_ssrf():
    connection = await client.get(joern_host, cpggen_host, joern_username, joern_password)
    await client.df(connection, {"parameter": "attacker-controlled"}, 'cpg.method.isExternal(true).where(_.typeDecl.fullName("java.util.Scanner")).parameter.order(1)')
asyncio.run(detect_ssrf())

src/main/java/com/example/vulnspring/WebController.java:119 @RequestParam(name = "dbpath") String dbpath
├── src/main/java/com/example/vulnspring/WebController.java:121 FilenameUtils.getBaseName(dbpath)
├── src/main/java/com/example/vulnspring/WebController.java:128 remoteUrl.openStream()
└── java.util.Scanner( p1 )

